In [373]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from pandas_summary import DataFrameSummary
from pathlib import Path

import os, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()
os.chdir('/home/krivas/projects/analysis-project/')
from src.utils import convert_int, convert_date,\
                        add_mean_features, add_date_diffs, add_days_count, add_date_features,\
                        make_set, run_model, apply_cats


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
DATA        = Path('data')
RAW         = DATA/'raw'
PROCESSED   = DATA/'processed'

# Reading data

In [303]:
csv = pd.read_csv(PROCESSED/'HistoricoJob.csv')

In [376]:
add_date_features(csv)

In [378]:
csv.columns

Index(['Id_HistoricoJob', 'Id_Job', 'Id_Malla', 'Fecha_Carga_Scheduler',
       'Fecha_Ejec_Inicio', 'Fecha_Ejec_Fin', 'Duracion', 'Promedio', 'Agente',
       'Mxrc', 'Maxcmpc', 'Grupo', 'Force_Complete', 'CCF', 'Estado',
       'duracion_int', 'promedio_int', 'Fecha_Ejec_Inicio_Int',
       'Hora_Ejec_Inicio_Int', 'Fecha_Ejec_Fin_Int', 'Hora_Ejec_Fin_Int',
       'DiaSemana'],
      dtype='object')

In [446]:
target_date = 20180521
start_date = 20180401

In [447]:
%%time
days_data = []
for date in range(start_date, target_date):
    if date in csv.Fecha_Ejec_Inicio_Int.unique():
        days_data.append(make_set(csv.loc[csv['Fecha_Ejec_Inicio_Int'] == date], csv.loc[csv['Fecha_Ejec_Inicio_Int'] == target_date]))

CPU times: user 1min 12s, sys: 52 ms, total: 1min 12s
Wall time: 1min 18s


# Preprocessing

In [441]:
from tqdm import tqdm

In [ ]:
%%time
for temp in tqdm(days_data):
    add_date_diffs(temp, csv)
    add_days_count(temp, csv)
    add_date_features(temp)
    add_mean_features(temp, csv, 3)

 28%|██▊       | 12/43 [05:13<13:28, 26.09s/it]

In [453]:
days_data[0].head()

,Fecha_Ejec_Inicio_Int,Id_Job,Id_Malla,duracion_int,DaysSinceMainframeOp,DaysCountMainframeOp,DiaSemana,promedio_por_dia,promedio_por_semana
0,20180401.0,@DF2LIAB,00FBFCL2,7.0,3,246,6,9,9
1,20180401.0,@DF2NO92,02FMIH92,0.0,1,593,6,0,0
2,20180401.0,@E0WA949,02WA192H,0.0,59,2,6,0,0
3,20180401.0,@E0WA969,02WA192H,0.0,59,2,6,0,0
4,20180401.0,@E2A6961,02A6H993,4.0,1,296,6,5,4


In [338]:
# cat_cols = ['DiaSemana']

In [339]:
# %%time
# for w in days_data:
#     for col in cat_cols:
#         w[col] = w[col].astype('category').cat.codes

CPU times: user 148 ms, sys: 0 ns, total: 148 ms
Wall time: 145 ms


# Model

In [454]:
#from src.utils import run_model
from lightgbm import LGBMRegressor

In [455]:
target_col = 'duracion_int'
pred_col = 'Prediccion'
id_cols = ['Fecha_Ejec_Inicio_Int', 'Id_Job', 'Id_Malla']
metric_names = ['mse']

In [456]:
days_data[len(days_data) - 1][pred_col] = 0

In [457]:
%%time
results = None
output = []
for i, w in enumerate(days_data[1:]):
    train, val, test = pd.concat(days_data[:i+1], axis=0), w, days_data[-1]
    
    X_train, y_train = train.drop(id_cols + [target_col], axis=1), \
                       train[target_col]
    if pred_col in val.columns: # when test acts as validation
        X_val, y_val = None, None
    else:
        X_val, y_val = val.drop(id_cols + [target_col], axis=1), \
                       val[target_col]
    X_test = test.drop(id_cols + [target_col, pred_col], axis=1)
    y_test, _, results, model = run_model(
            LGBMRegressor(n_estimators=120),
            X_train, y_train, X_val, y_val, X_test,
            metric_names, results,
            early_stopping=True)
    output.append([y_test, model])

[1]	valid_0's l2: 1.50461e+08
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's l2: 1.24972e+08
[3]	valid_0's l2: 1.04224e+08
[4]	valid_0's l2: 8.73288e+07
[5]	valid_0's l2: 7.35696e+07
[6]	valid_0's l2: 6.23403e+07
[7]	valid_0's l2: 5.3029e+07
[8]	valid_0's l2: 4.55026e+07
[9]	valid_0's l2: 3.92368e+07
[10]	valid_0's l2: 3.41654e+07
[11]	valid_0's l2: 3.00187e+07
[12]	valid_0's l2: 2.65239e+07
[13]	valid_0's l2: 2.3713e+07
[14]	valid_0's l2: 2.13344e+07
[15]	valid_0's l2: 1.94251e+07
[16]	valid_0's l2: 1.78499e+07
[17]	valid_0's l2: 1.64956e+07
[18]	valid_0's l2: 1.54151e+07
[19]	valid_0's l2: 1.4521e+07
[20]	valid_0's l2: 1.37372e+07
[21]	valid_0's l2: 1.31167e+07
[22]	valid_0's l2: 1.26182e+07
[23]	valid_0's l2: 1.21767e+07
[24]	valid_0's l2: 1.1809e+07
[25]	valid_0's l2: 1.1506e+07
[26]	valid_0's l2: 1.12411e+07
[27]	valid_0's l2: 1.0999e+07
[28]	valid_0's l2: 1.081e+07
[29]	valid_0's l2: 1.06491e+07
[30]	valid_0's l2: 1.05109e+07
[31]	valid_0's l2: 1.0392

In [458]:
model.feature_importances_

array([ 246, 1611,  403,  694,  646])

In [459]:
y_test = np.mean([x[0] for x in output], axis=0)

In [460]:
y_test

array([36.80700206, 88.04642465, 51.91097759, ..., 36.81533778,
       36.80700206, 36.80700206])

In [409]:
y_test

array([43.66349031, 73.23908373, 54.24019823, ..., 39.74926231,
       43.75155556, 43.75155556])

In [349]:
from sklearn.metrics import mean_squared_error

In [461]:
mean_squared_error(days_data[-1:][0].duracion_int, y_test)

482218.5862261524

In [333]:
mean_squared_error(days_data[-1:][0].duracion_int, y_test)

3287868.068427667

In [410]:
mean_squared_error(days_data[-1:][0].duracion_int, y_test)

552340.4452117465